In [1]:
%matplotlib inline
from __future__ import print_function

import numpy as np
import keras
from keras.models import Sequential, Model
from keras.optimizers import Adam, SGD
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import ResNet50
from keras.applications.xception import Xception
from keras.layers import Dense, Flatten, Dropout
from keras.layers import GlobalAveragePooling2D, BatchNormalization, Dense
from keras.optimizers import Adam, SGD, RMSprop
from keras.layers.advanced_activations import PReLU


Using TensorFlow backend.


In [2]:
train_path = 'data/train'
valid_path = 'data/valid'
batch_size = 32
target_size = 224

train_batches = ImageDataGenerator(rescale=1./ 255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True).flow_from_directory(train_path, target_size=(224, 224), batch_size=batch_size, class_mode='categorical')
valid_batches = ImageDataGenerator(rescale=1./ 255).flow_from_directory(valid_path, target_size=(224,224), batch_size=batch_size, class_mode='categorical')

Found 6889 images belonging to 120 classes.
Found 706 images belonging to 120 classes.


In [3]:
base_model = ResNet50(weights='imagenet', include_top=False)
#base_model = Xception(weights='imagenet', include_top=False, input_shape=(target_size, target_size, 3))

In [4]:
for layer in base_model.layers:
    layer.trainable = False

In [5]:
#output = base_model.output
#output = BatchNormalization()(output)
#output = GlobalAveragePooling2D()(output)
#output = Dropout(0.5)(output)
#output = Dense(1024, activation='relu')(output)
#output = Dropout(0.5)(output)
#predictions = Dense(120, activation='softmax', name='predictions')(output)

In [6]:
output = base_model.output
output = GlobalAveragePooling2D()(output)
output = Dropout(0.3)(output)
#output = Dense(1024, activation='relu')(output)
predictions = Dense(120, activation='softmax', name='predictions')(output)

In [7]:
#output = base_model.get_layer(index=-1).output
model = Model(base_model.input,predictions)

In [8]:
#optimizer = RMSprop(lr=0.001, rho=0.9)
optimizer = 'rmsprop'

In [9]:
#model.compile(SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True), loss='categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [10]:
model.fit_generator(train_batches, steps_per_epoch=200, validation_data=valid_batches, epochs=50)

Epoch 1/50


ValueError: Error when checking target: expected predictions to have shape (1,) but got array with shape (120,)

In [17]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
label_enc = LabelEncoder()
SEED=2018
labels = pd.read_csv('labels.csv')
np.random.seed(seed=SEED)
rnd = np.random.random(len(labels))
train_idx = rnd < 0.9
valid_idx = rnd >= 0.9
y_train = label_enc.fit_transform(labels["breed"].values)
ytr = y_train[train_idx]
yv = y_train[valid_idx]

In [18]:
labels

id                        breed
0      000bec180eb18c7604dcecc8fe0dba07                  boston_bull
1      001513dfcb2ffafc82cccf4d8bbaba97                        dingo
2      001cdf01b096e06d78e9e5112d419397                     pekinese
3      00214f311d5d2247d5dfe4fe24b2303d                     bluetick
4      0021f9ceb3235effd7fcde7f7538ed62             golden_retriever
5      002211c81b498ef88e1b40b9abf84e1d           bedlington_terrier
6      00290d3e1fdd27226ba27a8ce248ce85           bedlington_terrier
7      002a283a315af96eaea0e28e7163b21b                       borzoi
8      003df8b8a8b05244b1d920bb6cf451f9                      basenji
9      0042188c895a2f14ef64a918ed9c7b64           scottish_deerhound
10     004396df1acd0f1247b740ca2b14616e            shetland_sheepdog
11     0067dc3eab0b3c3ef0439477624d85d6                 walker_hound
12     00693b8bc2470375cc744a6391d397ec                  maltese_dog
13     006cc3ddb9dc1bd827479569fcdc52dc                     bluetick
14     0075dc49dab4024d12fafe67074d8a81              norfolk_terrier
15     00792e341f3c6eb33663e415d0715370          african_hunting_dog
16     007b5a16db9d9ff9d7ad39982703e429      wire-haired_fox_terrier
17     007b8a07882822475a4ce6581e70b1f8                      redbone
18     007ff9a78eba2aebb558afea3a51c469             lakeland_terrier
19     008887054b18ba3c7601792b6a453cc3                        boxer
20     008b1271ed1addaccf93783b39deab45                     doberman
21     008ba178d6dfc1a583617470d19c1673                   otterhound
22     009509be3ca7cce0ff9e37c8b09b1125                   otterhound
23     0097c6242c6f3071762d9f85c3ef1b2f           bedlington_terrier
24     00a338a92e4e7bf543340dc849230e75                        dingo
25     00a366d4b4a9bbb6c8a63126697b7656             golden_retriever
26     00a862390341c5be090dd72bd2bc19ef           standard_schnauzer
27     00b7d114bc5166a629a3cc03d9329120          irish_water_spaniel
28     00ba244566e36e0af3d979320fd3017f      black-and-tan_coonhound
29     00bee065dcec471f26394855c5c2f3de                        cairn
...                                 ...                          ...
10192  ff4afeb51a1473f7ba18669a8ff48bc9               border_terrier
10193  ff4bb57ce419cd637dd511a1b5474bff                  appenzeller
10194  ff52a3909f5801a71161cec95d213107  west_highland_white_terrier
10195  ff54d45962b3123bb67052e8e29a60e7                      basenji
10196  ff63ed894f068da8e2bbdfda50a9a9f8                        boxer
10197  ff63fa05a58473138848f80840064d23                affenpinscher
10198  ff6f47aa8e181b6efa4d0be7b09b5628             golden_retriever
10199  ff7334b06cee8667a7f30eb00e0b93cf                      whippet
10200  ff7d9c08091acc3b18b869951feeb013                  maltese_dog
10201  ff84992beff3edd99b72718bec9448d2           bedlington_terrier
10202  ff8e3fa7e04faca99af85195507ee54d             sealyham_terrier
10203  ff91c3c095a50d3d7f1ab52b60e93638                affenpinscher
10204  ffa0055ec324829882186bae29491645                  maltese_dog
10205  ffa0ad682c6670db3defce2575a2587f             english_springer
10206  ffa16727a9ee462ee3f386be865b199e              irish_wolfhound
10207  ffa4e1bf959425bad9228b04af40ac76                      basenji
10208  ffa6a8d29ce57eb760d0f182abada4bf             english_foxhound
10209  ffbbf7536ba86dcef3f360bda41181b4                   weimaraner
10210  ffc1717fc5b5f7a6c76d0e4ea7c8f93a                   great_dane
10211  ffc2b6b9133a6413c4a013cff29f9ed2                 irish_setter
10212  ffc532991d3cd7880d27a449ed1c4770              tibetan_terrier
10213  ffca1c97cea5fada05b8646998a5b788             english_foxhound
10214  ffcb610e811817766085054616551f9c                       briard
10215  ffcde16e7da0872c357fbc7e2168c05f                     airedale
10216  ffcffab7e4beef9a9b8076ef2ca51909                      samoyed
10217  ffd25009d635cfd16e793503ac5edef0                       borzoi
10218  ffd3f636f7f379c51ba3648a9ff8

In [23]:
for i in ytr:
    print(ytr[i])

10
0
68
66
73
73
83
7
93
25
17
68
88
60
3
42
49
110
118
73
0
66
57
79
67
19
97
56
90
57
66
10
90
42
14
37
0
19
77
42
106
15
14
49
14
60
57
19
46
5
95
97
18
57
118
5
10
48
89
74
73
26
80
88
21
26
10
93
58
68
115
70
48
38
48
90
58
17
89
65
57
29
80
60
65
42
49
118
24
49
60
80
33
57
93
46
118
48
33
0
0
5
49
26
0
68
46
46
58
70
73
10
73
66
118
71
68
55
77
37
19
42
21
5
90
49
90
37
71
67
5
40
69
33
83
114
73
113
57
67
118
90
77
93
94
115
17
15
95
90
83
106
4
49
74
10
49
106
0
68
80
72
24
15
25
79
19
68
53
56
88
110
56
19
70
38
56
29
95
7
56
89
19
14
55
110
70
24
114
12
114
90
49
49
15
114
97
15
118
17
73
114
37
6
99
48
93
77
93
93
17
10
77
113
48
77
68
85
16
16
18
68
48
15
85
27
33
37
0
24
72
88
37
21
38
15
69
42
37
97
42
57
70
15
68
6
90
77
5
40
65
3
49
7
15
49
68
60
114
24
90
97
42
24
73
77
73
7
110
33
10
0
15
19
5
90
110
68
0
19
56
114
46
58
14
36
49
77
10
49
67
21
97
115
118
40
25
90
55
24
88
10
60
114
60
55
66
36
25
68
36
68
94
97
15
49
106
67
8
10
37
37
84
93
69
83
48
118
24
118
49
11

3
72
25
88
37
58
21
37
94
115
90
118
17
16
38
42
65
18
7
95
68
3
15
97
14
40
57
93
49
54
14
10
37
37
42
17
110
68
65
95
73
40
10
19
17
57
2
114
4
53
113
73
97
118
5
48
66
17
0
54
21
37
68
56
88
26
46
40
90
25
49
90
12
38
27
4
0
3
94
90
94
18
118
17
89
57
83
115
72
24
90
72
90
0
48
18
48
4
84
21
16
99
73
15
21
115
40
114
49
106
65
57
57
95
56
48
37
93
15
72
88
49
15
27
88
95
68
16
90
68
95
70
115
37
57
40
88
56
118
48
118
114
24
55
53
18
10
8
106
89
90
66
40
58
15
16
3
97
3
0
49
68
21
48
19
68
53
118
88
93
90
49
21
74
7
97
5
106
38
89
6
89
90
5
110
66
57
40
69
33
19
118
49
115
49
46
57
12
10
65
118
15
69
94
114
40
48
19
37
49
90
25
54
49
36
97
99
53
88
25
25
0
37
79
7
106
118
57
49
4
49
58
42
15
97
15
58
49
21
58
37
21
49
5
106
97
24
58
5
37
37
69
67
40
69
0
42
60
24
58
85
37
48
80
37
24
3
26
19
55
79
106
21
14
21
74
77
73
19
46
80
16
10
66
15
0
3
8
90
114
3
68
118
3
36
73
99
37
97
56
83
58
18
24
72
90
37
118
49
3
37
29
88
114
46
7
5
37
77
94
90
5
73
56
42
15
18
95
118
3
68
58
15
17
0
1

69
48
97
37
26
58
93
16
21
42
67
0
12
40
10
33
42
114
97
24
57
37
93
68
29
0
57
90
56
26
15
37
29
57
90
114
37
88
57
33
0
57
68
48
48
118
84
15
48
38
49
88
69
88
37
6
42
5
58
77
19
4
77
21
90
106
19
33
90
56
90
77
42
10
71
40
49
33
37
89
3
49
65
18
29
0
6
95
90
55
56
48
56
118
25
115
118
89
14
90
36
54
77
0
16
27
7
93
37
0
84
40
49
74
118
93
70
68
58
14
80
77
7
21
5
69
71
67
99
94
58
93
90
69
18
118
38
8
49
99
49
21
94
84
110
15
77
46
42
2
85
14
40
19
66
70
77
97
93
48
71
74
14
67
73
90
5
89
54
58
25
85
0
15
48
80
49
2
36
56
37
90
94
5
77
67
66
57
97
77
88
15
83
74
2
15
17
95
48
40
3
97
6
77
77
33
21
74
90
42
15
10
68
40
4
53
69
56
10
88
69
65
89
58
57
80
37
24
69
48
54
8
37
84
118
40
69
89
89
57
49
49
26
115
90
24
40
74
5
74
93
49
72
84
73
60
58
58
57
56
42
69
65
70
25
37
89
106
49
90
14
95
37
58
88
66
69
106
24
118
79
19
68
15
54
114
77
68
77
27
10
7
84
66
6
77
40
5
115
49
56
40
42
118
8
18
18
67
115
14
57
58
24
37
15
90
113
10
90
14
90
72
37
15
40
48
68
56
73
57
77
26
42
25
10
0
73


97
27
77
3
58
29
90
48
25
106
14
70
58
49
5
110
69
118
33
65
77
24
89
15
60
36
42
93
56
42
56
90
14
49
93
114
65
77
66
0
118
95
115
15
73
71
53
97
114
89
25
36
18
68
80
38
10
85
7
49
19
66
27
17
73
69
19
17
2
94
7
68
90
8
5
68
12
49
83
36
114
114


In [20]:
yv

array([ 85, 113,   2, ..., 109,  94,   3])